In [1]:
#Importing Pandas library for cleaning up raw data
import pandas as pd
import numpy as np

In [2]:
#File name of raw csv, which should be in the same folder/directory as the Preprocessing Notebook
raw_filename = 'clean_data.csv'

In [9]:
data = pd.read_csv(raw_filename, sep='\t', index_col='Country')
data.head()

,Unnamed: 0,I consider myself:,WiFi Router:Check all the internet connected devices you currently own:,Laptop computer:Check all the internet connected devices you currently own:,Smart phone:Check all the internet connected devices you currently own:,Smart TV:Check all the internet connected devices you currently own:,Activity Tracker (ex: Fitbit or Apple Watch):Check all the internet connected devices you currently own:,"Smarthome Hub (ex. Amazon Echo, Google Alexa):Check all the internet connected devices you currently own:",Car that connects to the internet:Check all the internet connected devices you currently own:,Smart Thermostat (ex: Nest):Check all the internet connected devices you currently own:,"Smart Appliance (ex. Coffeemaker, Refrigerator, Oven, Fridge):Check all the internet connected devices you currently own:",Smart Door Locks (ex. Door locks for your home you can open via bluetooth):Check all the internet connected devices you currently own:,"Smart Lighting (ex. Connected lighting switches, dimmers, or bulbs):Check all the internet connected devices you currently own:",Thinking about a future in which so much of your world is connected to the internet leaves you feeling:,What are you most excited about as we move toward a more digitally connected future?,What is your biggest fear as we move towards a more connected future?,Total Owned Devices,Country Average Devices,Country Average Tech Proficiency,Country Average Tech Sentiment
Country,,,,,,,,,,,,,,,,,,,,
United States,0,3,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,-2,None of the above,The loss of privacy,7.0,3.800052,2.605362,-0.071605
United States,1,3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,-2,None of the above,We�۪ll lose touch with one another,6.0,3.800052,2.605362,-0.071605
United States,2,3,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,How much fun it will be,I have no fears about a more connected future,4.0,3.800052,2.605362,-0.071605
United States,3,3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,How it will bring the world together,The loss of privacy,3.0,3.800052,2.605362,-0.071605
Canada,4,3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,None of the above,The loss of privacy,3.0,3.513410,2.590333,-0.050987


In [10]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [11]:
from bokeh.io import show
from bokeh.plotting import figure

In [23]:
# plot country then I consider myself response
data.loc['United States']['I consider myself:']
p = figure(
            x_range=(1,4),
            y_range=(0,11),
)
p.circle(x=data.loc['United States']["I consider myself:"], y=data.loc['United States']['Total Owned Devices'])
show(p)

In [51]:
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models import LinearInterpolator

size_mapper = LinearInterpolator(
    x = [data['I consider myself:'].min(), data['I consider myself:'].max()],
    y=[5,50]
)

source = ColumnDataSource(
    dict(
        x=data.loc['United States']["I consider myself:"],
        y=data.loc['United States']['Total Owned Devices'],
        sentiment=data.loc['United States']['Thinking about a future in which so much of your world is connected to the internet leaves you feeling:']
    )
)
hover = HoverTool(tooltips = "@sentiment", show_arrow=False)
p = figure(
            title=str("Owned Objects by Sentiment for United States"),
            x_range=(0,5),
            y_range=(0,12),
            tools=[hover]
)
p.circle(
        x='x', y='y', 
        #size= {'field': 'I consider myself:','transform':size_mapper},
        source=source
)

show(p)


In [40]:
#Get column data source from data frame
source= ColumnDataSource(data.loc['United States'])
source.column_names
source.data['Total Owned Devices']

array([ 7.,  6.,  4., ...,  3.,  4.,  1.])

### Create variables for plotting

In [7]:
# x-axis values
x = ['2007','2008','2009','2010','2011','2012','2013','2014','2015']

#dictionary with the instituition column as the key and the values [0th index is Male Retention, 1th index is Female Retention 2nd index is Overall Retention]
plotDictionary = {}

def checkNaN(raw_val):
    if (pd.isnull(raw_val)):
        return float('nan')
    else:
        return float((1-raw_val)*100)

# create y values for (Male, Female, Overall Retention) for each school
for inst in institutions:
    #set 0 array for start    
    mRetention = 9*[0]
    fRetention = 9*[0]
    oRetention = 9*[0]
    
    #get rows of data for each inst
    for year in x:
        #print("searching for year %s") %year
        for i,row in new_dataframe.loc[new_dataframe['Institution'].isin([inst]),:].iterrows():     
            if(int(year) == int(new_dataframe.loc[i]["School Year"][:4] )):
                #print(new_dataframe.loc[i,'Institution'])
                if(pd.isnull(new_dataframe.loc[i]["Male Retention"])):
                    mRetention[x.index(year)] = float('nan')
                    fRetention[x.index(year)] = float('nan')
                    oRetention[x.index(year)] = float('nan')
                    break
                else:
                    mRetention[x.index(year)] = checkNaN(new_dataframe.loc[i]["Male Retention"])
                    fRetention[x.index(year)] = checkNaN(new_dataframe.loc[i]["Female Retention"])
                    oRetention[x.index(year)] = checkNaN(new_dataframe.loc[i]["Overall Retention"])
                    break
            #else:
                #print("cant find %s") %year
    #add retention % as value with institution as its key
    plotDictionary[inst] = mRetention, fRetention, oRetention
    

## Graphing Data

### Importing libraries

In [8]:
from ipywidgets import interact
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import Category10
from bokeh.layouts import gridplot
import itertools
from bokeh.models import ColumnDataSource
from bokeh.embed import file_html


output_notebook()

Loading BokehJS ...

In [9]:
# Method for cycling through color pallete
def color_gen():
    for c in itertools.cycle(Category10[10]):
        yield c
color = color_gen()

In [10]:
# #create a new plot
# plot1 = figure(
#     title="Retention Rates Among CS Students",
#     x_axis_label='Year', y_axis_label='Retention %',
#     y_range=(-5,5)
# )

# plot2 = figure(
#     title="Retention Rates Among CS Students",
#     x_axis_label='Year', y_axis_label='Retention %',
#     y_range=(-5,5)
# )

#draw lines
def drawLines(retentionFilter, inst):
    titleText=""
    if retentionFilter == 0: titleText='(Male)'    
    elif retentionFilter == 1: titleText='(Female)'
    elif retentionFilter == 2: titleText='(Male and Female)'
        
    ## Graph showing all retention rates for all filtered institutions    
    plot1 = figure(
        title="Retention Rates Among CS Students "+titleText,
        x_axis_label='Year', y_axis_label='Retention %',
        y_range=(-500,500)    
    )
    
    for institution in plotDictionary.keys():
        y = plotDictionary[institution][retentionFilter]   
        plot1.line(x, y, legend=str(institution), line_width=2, color=next(color))
    
    ## Graph showing all retention rates for selected institutions
    plot2 = figure(
    title="Retention Rates Among CS Students "+ titleText +" for Institution "+str(inst) ,
        x_axis_label='Year', y_axis_label='Retention %',
        y_range=(-500,500)
    )

    y2 = plotDictionary[inst][retentionFilter]   
    plot2.line(x, y2, legend=str(inst), line_width=2, color=next(color))

    plots = gridplot([[plot1,plot2]])
    show(plots, notebook_handle=True)   

In [11]:
#callback method that hanldes graph updateswhen a change is made to the widget 
def update(FilterGender, ViewInstitution):
    if   FilterGender == "Male Retention": 
        drawLines(0, float(inst))
    elif FilterGender == "Female Retention": 
        drawLines(1,float(inst))
    elif FilterGender == "Overall Retention": 
        drawLines(2, float(inst))
    push_notebook()    

In [12]:
# Draw Graphs and control Widgets
interact(update, FilterGender=["Male Retention", "Female Retention", "Overall Retention"], ViewInstitution= map(str, institutions))
html = file_html(plots, CDN, "my plot")

A Jupyter Widget

NameError: name 'plots' is not defined